# Claude Tool Schema Generator

This notebook demonstrates how to use the `claude_schema_generator` utility to automatically generate Claude tool use JSON schemas from Python function definitions.

## Features

- Parse function signatures, type hints, and docstrings
- Support for Google, NumPy, and Sphinx docstring styles
- Handle Union types, Optional parameters, and complex types
- Generate proper Claude tool schema format
- Validate output schemas

In [2]:
# Import the schema generator
import sys
sys.path.append('../utils')

from claude_schema_generator import (
    generate_tool_schema, 
    generate_schema_from_function,
    validate_schema
)
import json
from typing import List, Optional, Dict, Union

## Example 1: Simple Function with Google-style Docstring

In [ ]:
def calculate_area(length: float, width: float, unit: str = "meters") -> float:
    """
    Calculate the area of a rectangle
    
    Args:
        length (float): The length of the rectangle
        width (float): The width of the rectangle  
        unit (str): The unit of measurement. Defaults to "meters".
        
    Returns:
        float: The calculated area
    """
    return length * width

# Generate schema
schema = generate_schema_from_function(calculate_area)
print("Generated Schema:")
print(json.dumps(schema, indent=2))

## Example 2: Complex Function with Optional and List Parameters

In [ ]:
def create_user_profile(name: str, age: int, email: Optional[str] = None, 
                       tags: List[str] = None, metadata: Dict[str, str] = None) -> Dict:
    """
    Create a user profile with the given information
    
    Args:
        name (str): The user's full name
        age (int): The user's age in years
        email (str, optional): The user's email address
        tags (List[str], optional): List of tags associated with the user
        metadata (Dict[str, str], optional): Additional metadata as key-value pairs
        
    Returns:
        Dict: The created user profile
    """
    profile = {"name": name, "age": age}
    if email:
        profile["email"] = email
    if tags:
        profile["tags"] = tags
    if metadata:
        profile["metadata"] = metadata
    return profile

schema = generate_schema_from_function(create_user_profile)
print("Generated Schema:")
print(json.dumps(schema, indent=2))

## Example 3: Function with NumPy-style Docstring

In [ ]:
def search_database(query: str, limit: int = 10, filters: Optional[Dict] = None) -> List[Dict]:
    """
    Search the database with the given query and filters
    
    Parameters
    ----------
    query : str
        The search query string
    limit : int
        Maximum number of results to return
    filters : Dict, optional
        Additional filters to apply to the search
        
    Returns
    -------
    List[Dict]
        List of matching database records
    """
    # Mock implementation
    return [{"id": 1, "title": "Sample Result"}]

schema = generate_schema_from_function(search_database)
print("Generated Schema:")
print(json.dumps(schema, indent=2))

## Example 4: Generate Schema from Function Definition String

In [ ]:
# You can also generate schemas from function definition strings
function_def = '''
def send_email(to: str, subject: str, body: str, cc: Optional[List[str]] = None, 
               priority: str = "normal") -> bool:
    """
    Send an email to the specified recipients
    
    Args:
        to (str): Primary recipient email address
        subject (str): Email subject line
        body (str): Email body content
        cc (List[str], optional): List of CC recipients
        priority (str): Email priority level (low, normal, high)
        
    Returns:
        bool: True if email was sent successfully
    """
    return True
'''

schema = generate_tool_schema(function_def)
print("Generated Schema:")
print(json.dumps(schema, indent=2))

## Example 5: Validate Generated Schemas

In [ ]:
# Test schema validation
def simple_function(message: str) -> str:
    """A simple function for testing"""
    return f"Hello, {message}!"

schema = generate_schema_from_function(simple_function)
is_valid = validate_schema(schema)

print(f"Schema is valid: {is_valid}")
print("\nSchema:")
print(json.dumps(schema, indent=2))

## Example 6: Generate Schema for Existing Claude Helper Functions

In [ ]:
# Import our existing Claude helper functions
from claude_helpers import simple_chat, chat

# Generate schema for simple_chat function
schema = generate_schema_from_function(simple_chat)
print("Schema for simple_chat:")
print(json.dumps(schema, indent=2))

print("\n" + "="*50 + "\n")

# Generate schema for chat function
schema = generate_schema_from_function(chat)
print("Schema for chat:")
print(json.dumps(schema, indent=2))

## Example 7: Using the CLI Script

In [ ]:
# Demonstrate CLI usage (run these in terminal)
print("CLI Usage Examples:")
print("")
print("# List all functions in a file:")
print("python ../generate_schema.py ../utils/claude_helpers.py --list")
print("")
print("# Generate schema for a specific function:")
print("python ../generate_schema.py ../utils/claude_helpers.py simple_chat")
print("")
print("# Generate and save to file:")
print("python ../generate_schema.py ../utils/claude_helpers.py simple_chat --output simple_chat_schema.json")
print("")
print("# Generate with validation:")
print("python ../generate_schema.py ../utils/claude_helpers.py simple_chat --validate")

In [ ]:
# Let's actually run the CLI to list functions
import subprocess
import os

# Change to parent directory to run the CLI script
os.chdir('..')

# List functions in claude_helpers.py
result = subprocess.run(
    ['python', 'generate_schema.py', 'utils/claude_helpers.py', '--list'],
    capture_output=True, text=True
)

print("CLI Output:")
print(result.stdout)
if result.stderr:
    print("Errors:")
    print(result.stderr)

In [ ]:
# Generate schema using CLI
result = subprocess.run(
    ['python', 'generate_schema.py', 'utils/claude_helpers.py', 'simple_chat', '--validate'],
    capture_output=True, text=True
)

print("Generated Schema via CLI:")
print(result.stdout)
if result.stderr:
    print("\nStatus Messages:")
    print(result.stderr)

## Example 8: Real-world Tool Function

In [ ]:
def analyze_sentiment(text: str, model: str = "basic", 
                     return_confidence: bool = False) -> Union[str, Dict[str, float]]:
    """
    Analyze the sentiment of the given text
    
    This function analyzes text sentiment using various models and can return
    either a simple sentiment label or detailed confidence scores.
    
    Args:
        text (str): The text to analyze for sentiment
        model (str): The sentiment analysis model to use (basic, advanced, neural)
        return_confidence (bool): Whether to return confidence scores along with sentiment
        
    Returns:
        Union[str, Dict[str, float]]: Either sentiment label or dict with scores
    """
    # Mock implementation
    if return_confidence:
        return {"positive": 0.8, "negative": 0.1, "neutral": 0.1}
    else:
        return "positive"

schema = generate_schema_from_function(analyze_sentiment)
print("Schema for sentiment analysis tool:")
print(json.dumps(schema, indent=2))

print(f"\nSchema validation: {'PASSED' if validate_schema(schema) else 'FAILED'}")

## Summary

The Claude Schema Generator provides:

1. **Automatic schema generation** from Python functions
2. **Multiple docstring format support** (Google, NumPy, Sphinx)
3. **Type hint parsing** with proper JSON Schema type mapping
4. **CLI interface** for easy integration into workflows
5. **Schema validation** to ensure Claude compatibility
6. **Flexible usage** - from function objects or definition strings

This makes it dead simple to convert any Python function into a proper Claude tool schema!